# Exploratory Data Analysis

## Introduction
This report details the Exploratory Data Analysis (EDA) of a transactional dataset from a UK-based online retail company specializing in unique all-occasion gifts. The data covers transactions from December 1, 2010, to December 9, 2011. The primary objective of this EDA is to understand the dataset's structure, identify data quality issues, and uncover initial insights into sales patterns, product performance, and customer behavior. This analysis will serve as the strategic blueprint for developing a scalable data pipeline and a business intelligence dashboard to track key performance indicators (KPIs).

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
import openpyxl

In [22]:
df = pd.read_excel('Online Retail.xlsx')

## 1. Initial Data Inspection
This steps is to understanding data type and intergerity
- inspect data type
- Missing Value
- Duplicate Rows
- Initial Summary

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Notes:
1. This dataset have 541909 rows and 8 columns.
2. The InvoiceDate column is already in pandas datetime format, so no further conversion is needed. The Quantity and UnitPrice columns are continuous numerical variables with appropriate data types (int64 and float64).
3. Other categorical data still in the Object type data type. Object data type is fundamental data type that all data structure basically fit in this type. better change it to string since these columns are categorical data.
4. almost all column have no null value except CustomerID. This column have more than 100k null so drop all of them will result significant data loss. better let null value in these column and adjust data analysis letter.

In [24]:
# Drop Duplicate Value
df.drop_duplicates(inplace=True)

In [25]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,536641.000000,536641,536641.000000,401604.000000
mean,9.620029,2011-07-04 08:57:06.087421952,4.632656,15281.160818
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 10:52:00,1.250000,13939.000000
50%,3.000000,2011-07-19 14:04:00,2.080000,15145.000000
75%,10.000000,2011-10-18 17:05:00,4.130000,16784.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,219.130156,NaN,97.233118,1714.006089


Note:
1. There are negative values in quantity and Unit_prico column.
2. possibly there are outlier in quantity and unit price column because in quantity median is 3, 75 percentile is 10 but max value is 80995. there is high possibility that this is outlier. we need to mark outlier

##  2. Individual Column Analysis (Univariate Analysis)

In [26]:
# Inspect Negative value and Canceled Invoice
df[df['Quantity'] < 0][['InvoiceNo','Quantity']].sample(50)

,InvoiceNo,Quantity
304678,C563600,-1
229336,557031,-65
125231,C547031,-8
228905,C556966,-1
263160,C559939,-10
25554,C538373,-12
221959,556299,-161
122409,C546870,-2
387919,C570406,-1
44305,C540164,-2


In [27]:
df[df['Quantity'] < 0]['Quantity'].count()

np.int64(10587)

In [28]:
df['InvoiceNo'].str.startswith('C').count()

np.int64(9254)

In [29]:
df[(df['Quantity'] < 0) & (df['InvoiceNo'].str.startswith('C'))][['InvoiceNo', 'Quantity']]

,InvoiceNo,Quantity
141,C536379,-1
154,C536383,-1
235,C536391,-12
236,C536391,-24
237,C536391,-24
...,...,...
540449,C581490,-11
541541,C581499,-1
541715,C581568,-5
541716,C581569,-1


Note:

1. There are about 10,000 negative values in the Quantity column. Upon inspection, most of these negatives correspond to cancellations or returns, which are typically marked by InvoiceNos starting with 'C'. 
2. However, there are over 1,000 negative quantity entries without corresponding cancellations (i.e., their InvoiceNo does not start with 'C'). These might be data entry errors or anomalies that need further investigation or cleaning.

Action:
1. Cancellation and Negative Values Are Important

- Negative quantities often represent cancellations or returns, which are critical for understanding customer behavior, calculating customer worth accurately, and determining total revenue.
- Dropping all negative values indiscriminately would overestimate revenue and may bias any customer segmentation or lifetime value calculations.
- Keep all negative values with InvoiceNo starting with 'C' because they properly represent valid cancellations or returns.

2. Data Entry Errors or Anomalies Should Be Removed

- Negative quantities that are not linked to cancellation invoices (i.e., InvoiceNo not starting with 'C') are possible data entry errors.
- Since these represent a small fraction of the dataset and their cause or correction can't be confidently inferred, it’s reasonable to drop these rows.
- Document the removal decision clearly to maintain transparency and reproducibility.


In [30]:
# Inspect Description column
df['Description'].value_counts().reset_index()

,Description,count
0,WHITE HANGING HEART T-LIGHT HOLDER,2357
1,REGENCY CAKESTAND 3 TIER,2189
2,JUMBO BAG RED RETROSPOT,2156
3,PARTY BUNTING,1720
4,LUNCH BAG RED RETROSPOT,1625
...,...,...
4218,LIGHT DECORATION BATTERY OPERATED,1
4219,RUBY GLASS CLUSTER NECKLACE,1
4220,ANT SILVER TURQUOISE BOUDICCA RING,1
4221,water damaged,1


Notes: there are 4223 unique products

In [31]:
# Top 10 Best Seller Product
df.groupby(['Description'])['Quantity'].sum().sort_values(ascending=False).head(10).reset_index()

,Description,Quantity
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53751
1,JUMBO BAG RED RETROSPOT,47260
2,POPCORN HOLDER,36322
3,ASSORTED COLOUR BIRD ORNAMENT,36282
4,PACK OF 72 RETROSPOT CAKE CASES,36016
5,WHITE HANGING HEART T-LIGHT HOLDER,35298
6,RABBIT NIGHT LIGHT,30631
7,MINI PAINT SET VINTAGE,26437
8,PACK OF 12 LONDON TISSUES,26299
9,PACK OF 60 PINK PAISLEY CAKE CASES,24719


In [32]:
# Inspect Customer
df['CustomerID'].unique()

array([17850., 13047., 12583., ..., 13298., 14569., 12713.], shape=(4373,))

There are 4373 unique customer

In [ ]:
# Inspect Country
df.groupby(['Country'])['InvoiceNo'].count().sort_values(ascending=False).head(10)

Country
United Kingdom    490300
Germany             9480
France              8541
EIRE                8184
Spain               2528
Netherlands         2371
Belgium             2069
Switzerland         1994
Portugal            1510
Australia           1258
Name: InvoiceNo, dtype: int64

UK is top country by order count

In [36]:
# Inspect unique customer count each country
df.groupby(['Country', 'CustomerID']).count().reset_index().groupby('Country')['CustomerID'].count().sort_values(ascending=False)

Country
United Kingdom          3950
Germany                   95
France                    87
Spain                     31
Belgium                   25
Switzerland               21
Portugal                  19
Italy                     15
Finland                   12
Austria                   11
Norway                    10
Australia                  9
Denmark                    9
Netherlands                9
Channel Islands            9
Japan                      8
Sweden                     8
Cyprus                     8
Poland                     6
Unspecified                4
USA                        4
Canada                     4
Israel                     4
Greece                     4
EIRE                       3
United Arab Emirates       2
Bahrain                    2
Malta                      2
Brazil                     1
Lebanon                    1
Iceland                    1
Czech Republic             1
European Community         1
Saudi Arabia               1
RSA   

# Summary of Key EDA Findings
- The initial analysis of the dataset, which contains 541,909 transactions, revealed several key characteristics and data quality issues that must be addressed:


- Data Integrity: The dataset contains a significant number of duplicate rows, which were removed. There are also substantial missing values, most critically in the 

- CustomerID column, where over 135,000 entries are null. An effective customer analysis, such as an RFM model, will require handling these missing customer identifiers.


- Cancellations and Returns: The data includes over 10,000 transactions with negative Quantity values, indicating returns or cancellations. The majority of these are correctly marked with an 

- InvoiceNo starting with 'C'. However, a notable subset of over 1,000 transactions have negative quantities without this cancellation code, suggesting data entry errors that must be filtered out.



- Sales Outliers: The Quantity and UnitPrice columns exhibit extreme outliers. For instance, the maximum quantity is 80,995, while the 75th percentile is only 10. This is likely due to the company's business model, which includes serving wholesale customers. A segmentation strategy will be required to analyze standard retail and wholesale orders separately.



- Initial Insights: The United Kingdom is, by a large margin, the primary market both in order volume and unique customers. Product-level analysis shows a high concentration of sales around specific items like "JUMBO BAG RED RETROSPOT" and "WHITE HANGING HEART T-LIGHT HOLDER".